In [1]:
# after uplading the humour-detection.zip and data.zip files
!unzip -o humor-detection.zip
!unzip -o data.zip

Archive:  humor-detection.zip
   creating: humor-detection/
  inflating: humor-detection/pyproject.toml  
   creating: humor-detection/humor_detection/
  inflating: humor-detection/humor_detection/decoder.py  
  inflating: humor-detection/humor_detection/test.py  
  inflating: humor-detection/humor_detection/train.py  
  inflating: humor-detection/humor_detection/utils.py  
 extracting: humor-detection/humor_detection/__init__.py  
  inflating: humor-detection/humor_detection/predict.py  
  inflating: humor-detection/humor_detection/dataset.py  
  inflating: humor-detection/humor_detection/encoder.py  
Archive:  data.zip
   creating: data/
  inflating: data/test.csv           
   creating: data/raw/
  inflating: data/raw/stupid_stuff.json  
  inflating: data/raw/spanish_jokes.csv  
  inflating: data/raw/haha_train.csv  
  inflating: data/raw/haha_test.csv  
   creating: data/classification/
  inflating: data/classification/test.csv  
  inflating: data/classification/train.csv  
  infla

In [2]:
%%capture
!pip install ./humor-detection
!pip install --upgrade transformers datasets bitsandbytes
!pip uninstall wandb -y

In [2]:
from humor_detection.encoder import (
    classification_model,
    detection_model,
    load_model,
    save_model,
)
from humor_detection.test import test_classification, test_detection
from humor_detection.train import train_classification, train_detection
from humor_detection.predict import predict_classification, predict_detection
from transformers.training_args import TrainingArguments

FULL_DATA_TEST = True
EPOCHS = 3
BATCH_SIZE = 16
GRAD_ACC_STEPS = 4

# Nombre del modelo en HuggingFace
model_owner = "google"
model_name = "canine-c"
hf_model_id = f"{model_owner}/{model_name}"
# Ajustes de trainer de Transformers https://huggingface.co/docs/transformers/v4.51.3/en/main_classes/trainer#transformers.TrainingArguments
#  Lo más importante es usar bf16 o fp16 para VRAM, batch_sizes para la velocidad y train_epochs para los epochs
arguments = TrainingArguments(
    bf16=True,
    bf16_full_eval=True,
    eval_strategy="epoch",
    num_train_epochs=EPOCHS,
    optim="adamw_8bit",
    per_device_eval_batch_size=BATCH_SIZE,
    per_device_train_batch_size=BATCH_SIZE,
    save_strategy="no",
    gradient_accumulation_steps=GRAD_ACC_STEPS,
    gradient_checkpointing=True,
)
# Generar el modelo acepta el mismo LoRA usado en PLN  https://huggingface.co/docs/transformers/v4.51.3/en/peft#peft
# lora = LoraConfig("CAUSAL_LM", lora_alpha=16, lora_dropout=0.1, r=128)

# Prompts por si quieren verificar algo manualmente
prompts = [
    "¿Cuál es el último animal que subió al arca de Noé? El del-fin.",
    "El otro día unas chicas llamarón a mi puerta y me pidieron una pequeña donación para una piscina local.\nLes di un garrafa de agua.",
    "The brain surgeon changed my life. He really opened my mind.",
    "djasndoasndoa",
    "jajaja",
]

# Tarea de clasificación 1 a 5 (Los labels son 0 a 4).
def run_classification():
    # Función para crear el modelo, tokenizador y añadir un lora si es necesario
    model, tokenizer = classification_model(hf_model_id)
    # Entrenamiento con datos en español, Con full_dataset=True entrenan el modelo final, english_data=True añade el dataset en inglés
    # y prompter permite usar una función para añadir modificar el texto de cada chiste
    train_classification(model, tokenizer, arguments, full_dataset=FULL_DATA_TEST)
    # Recolección de datos de test con dataset hecho por nosotros
    print(test_classification(model, tokenizer, arguments))
    path = f"./models/{model_name}/classification"
    # Función para guardar el modelo
    save_model(model, path)
    # Función para cargar el modelo
    model, _ = load_model(hf_model_id, path)
    # Predicción manual de prompts
    print(predict_classification(model, tokenizer, prompts, arguments))

# Tarea de detección 0 o 1.
def run_detection():
    model, tokenizer = detection_model(hf_model_id)
    train_detection(model, tokenizer, arguments, full_dataset=FULL_DATA_TEST)
    print(test_detection(model, tokenizer, arguments))
    path = f"./models/{model_name}/detection"
    save_model(model, path)
    model, _ = load_model(hf_model_id, path)
    print(predict_detection(model, tokenizer, prompts, arguments))

In [3]:
run_classification()
run_detection()

Some weights of CanineForSequenceClassification were not initialized from the model checkpoint at google/canine-c and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/9253 [00:00<?, ? examples/s]

Map:   0%|          | 0/1178 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,0,1,2,3,4,Accuracy,Macro avg,Weighted avg
1,1.167900,1.290425,"{'precision': 1.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 520.0}","{'precision': 0.30475382003395585, 'recall': 1.0, 'f1-score': 0.4671437865972674, 'support': 359.0}","{'precision': 1.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 222.0}","{'precision': 1.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 65.0}","{'precision': 1.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 12.0}",0.304754,"{'precision': 0.8609507640067913, 'recall': 0.2, 'f1-score': 0.09342875731945348, 'support': 1178.0}","{'precision': 0.7881210707913329, 'recall': 0.30475382003395585, 'f1-score': 0.1423638534706443, 'support': 1178.0}"
2,1.156000,1.343737,"{'precision': 1.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 520.0}","{'precision': 0.30475382003395585, 'recall': 1.0, 'f1-score': 0.4671437865972674, 'support': 359.0}","{'precision': 1.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 222.0}","{'precision': 1.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 65.0}","{'precision': 1.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 12.0}",0.304754,"{'precision': 0.8609507640067913, 'recall': 0.2, 'f1-score': 0.09342875731945348, 'support': 1178.0}","{'precision': 0.7881210707913329, 'recall': 0.30475382003395585, 'f1-score': 0.1423638534706443, 'support': 1178.0}"
3,1.153800,1.337261,"{'precision': 1.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 520.0}","{'precision': 0.30475382003395585, 'recall': 1.0, 'f1-score': 0.4671437865972674, 'support': 359.0}","{'precision': 1.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 222.0}","{'precision': 1.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 65.0}","{'precision': 1.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 12.0}",0.304754,"{'precision': 0.8609507640067913, 'recall': 0.2, 'f1-score': 0.09342875731945348, 'support': 1178.0}","{'precision': 0.7881210707913329, 'recall': 0.30475382003395585, 'f1-score': 0.1423638534706443, 'support': 1178.0}"


Trainer is attempting to log a value of "{'precision': 1.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 520.0}" of type <class 'dict'> for key "eval/0" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.30475382003395585, 'recall': 1.0, 'f1-score': 0.4671437865972674, 'support': 359.0}" of type <class 'dict'> for key "eval/1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 1.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 222.0}" of type <class 'dict'> for key "eval/2" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 1.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 65.0}" of type <class 'dict'> for key "eval/3" as a scalar. This invocation of Tensorb

...

Map:   0%|          | 0/1991 [00:00<?, ? examples/s]

Trainer is attempting to log a value of "{'precision': 0.8261562998405104, 'recall': 0.6371463714637147, 'f1-score': 0.7194444444444444, 'support': 813.0}" of type <class 'dict'> for key "eval/0" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.783724340175953, 'recall': 0.9074702886247877, 'f1-score': 0.8410700236034618, 'support': 1178.0}" of type <class 'dict'> for key "eval/1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8049403200082317, 'recall': 0.7723083300442513, 'f1-score': 0.7802572340239531, 'support': 1991.0}" of type <class 'dict'> for key "eval/macro avg" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8010509013046748, '

{'eval_loss': 0.5146138668060303, 'eval_model_preparation_time': 0.0039, 'eval_0': {'precision': 0.8261562998405104, 'recall': 0.6371463714637147, 'f1-score': 0.7194444444444444, 'support': 813.0}, 'eval_1': {'precision': 0.783724340175953, 'recall': 0.9074702886247877, 'f1-score': 0.8410700236034618, 'support': 1178.0}, 'eval_accuracy': 0.797086891009543, 'eval_macro avg': {'precision': 0.8049403200082317, 'recall': 0.7723083300442513, 'f1-score': 0.7802572340239531, 'support': 1991.0}, 'eval_weighted avg': {'precision': 0.8010509013046748, 'recall': 0.797086891009543, 'f1-score': 0.7914057363828284, 'support': 1991.0}, 'eval_runtime': 46.0493, 'eval_samples_per_second': 43.236, 'eval_steps_per_second': 2.714}


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

tensor([1, 1, 0, 0, 0])
